Recurrent Mask R-CNN Demo

Ref: https://github.com/cechung/R-RCNN

- Single Object Tracking
- detection module: Matterport implementation Mask R-CNN
- tracking module: 2 LSTM networks

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# need tf 1.15.2
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [4]:
# fix AttributeError: 'str' object has no attribute 'decode'
!pip install h5py==2.10.0

In [5]:
# fix AttributeError: module 'scipy.misc' has no attribute 'imresize'
!pip install scipy==1.2.2

In [6]:
# go to folder containing LSTM_Mask_RCNN repo and outputs
import os
root = "/content/drive/MyDrive/data/summer-project/LSTM_Mask_RCNN/"
os.chdir(root)
!pwd

/content/drive/MyDrive/data/summer-project/LSTM_Mask_RCNN


In [7]:
# install requirements
!pip install -r requirements.txt

# Training

In [8]:
# use pre-trained weight kaggle_bowl.h5

# Inference

## inference demos of Mask R-CNN and Mask R-CNN + LSTM

In [9]:
# demo mask r-cnn

# detect images in demo_video_nucleus/frames.

# The single image in demo_video_nucleus/frames/stage1_test/0000 can be detected using
# nucleus.py from https://github.com/matterport/Mask_RCNN/tree/master/samples/nucleus
# but is not detected using demo_mrcnn.py
!python demo_mrcnn.py

Using TensorFlow backend.

Configurations:
BACKBONE_SHAPES                [[160 160]
 [ 80  80]
 [ 40  40]
 [ 20  20]
 [ 10  10]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  640
IMAGE_MIN_DIM                  480
IMAGE_PADDING                  True
IMAGE_SHAPE                    [640 640   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_

In [ ]:
# new script using nucleus.py from https://github.com/matterport/Mask_RCNN/tree/master/samples/nucleus
'''
# go to samples/nucleus folder in LSTM_Mask_RCNN repo
import os
root = "/content/drive/MyDrive/data/summer-project/LSTM_Mask_RCNN/samples/nucleus"
os.chdir(root)
!pwd
# import nucleus is only available in samples/nucleus directory
import nucleus

import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")

# Dataset directory
# DATASET_DIR = os.path.join(ROOT_DIR, "single-particle-dataset/nucleus")
DATASET_DIR = os.path.join(ROOT_DIR, "demo_video_nucleus/frames")

# Inference Configuration
config = nucleus.NucleusInferenceConfig()
config.display()

# Device to load the neural network on.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# Only inference mode is supported right now
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    fig.tight_layout()
    return ax

# Load validation dataset
dataset = nucleus.NucleusDataset()
dataset.load_nucleus(DATASET_DIR, "stage1_test")

dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference",
                              model_dir=LOGS_DIR,
                              config=config)

# Path to a specific weights file
weights_path = "../../kaggle_bowl.h5"

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))
print("Original image shape: ", modellib.parse_image_meta(image_meta[np.newaxis,...])["original_image_shape"][0])

# print("object detection using detect_molded()")
# # Run object detection
# results = model.detect_molded(np.expand_dims(image, 0), np.expand_dims(image_meta, 0), verbose=1)

# # Display results
# r = results[0]
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

# # Compute AP over range 0.5 to 0.95 and print it
# utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
#                        r['rois'], r['class_ids'], r['scores'], r['masks'],
#                        verbose=1)

# visualize.display_differences(
#     image,
#     gt_bbox, gt_class_id, gt_mask,
#     r['rois'], r['class_ids'], r['scores'], r['masks'],
#     dataset.class_names, ax=get_ax(),
#     show_box=False, show_mask=False,
#     iou_threshold=0.5, score_threshold=0.5)


# regular detection function
print("object detection using detect()")
img_name = str(image_id) + ".png" # ".jpg"
import utils.utils as utils
import utils.visualize as visualize
coco_class_names = ['BG', 'cell']
RESULT_DIR = os.path.join(ROOT_DIR, "mrcnn_result")
regular_results = model.detect([image], verbose=1)
reg_r = regular_results[0]
# print("reg r:", reg_r)
# Visualize results
visualize.save_image(image, img_name, reg_r['rois'], reg_r['masks'],
  reg_r['class_ids'], reg_r['scores'], coco_class_names,
  filter_classs_names=None, scores_thresh=0.1, 
  save_dir=RESULT_DIR, mode=1)

print("Done")
'''

'\n# go to samples/nucleus folder in LSTM_Mask_RCNN repo\nimport os\nroot = "/content/drive/MyDrive/data/summer-project/LSTM_Mask_RCNN/samples/nucleus"\nos.chdir(root)\n!pwd\n# import nucleus is only available in samples/nucleus directory\nimport nucleus\n\nimport os\nimport sys\nimport random\nimport math\nimport re\nimport time\nimport numpy as np\nimport tensorflow as tf\nimport matplotlib\nimport matplotlib.pyplot as plt\nimport matplotlib.patches as patches\n\n# Root directory of the project\nROOT_DIR = os.path.abspath("../../")\n\n# Import Mask RCNN\nsys.path.append(ROOT_DIR)  # To find local version of the library\nfrom mrcnn import utils\nfrom mrcnn import visualize\nfrom mrcnn.visualize import display_images\nimport mrcnn.model as modellib\nfrom mrcnn.model import log\n\n%matplotlib inline \n\n# Directory to save logs and trained model\nLOGS_DIR = os.path.join(ROOT_DIR, "logs")\n\n# Dataset directory\n# DATASET_DIR = os.path.join(ROOT_DIR, "single-particle-dataset/nucleus")\nD

In [ ]:
# demo r-rcnn
!python demo_rrcnn.py

Using TensorFlow backend.

Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                2.0
IMAGE_RESIZE_MODE              pad64
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14